In [1]:
import json
import geopy.distance
import numpy as np
import itertools
from itertools import chain
from itertools import combinations
import gurobipy as gp
from gurobipy import GRB
import folium
import random
import math

# Part I: Megan's First Day Driving Uber Eats

### Functions to get data in correct form for model: 

In [2]:
#### Function: Powerset ####
## Input: any list 
## Return: all possible subsets of list
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


#### Function: findsubsetsL2 ####
## Input: any list
## Return: All subsets of size two
def findsubsetsL2(s):
    return list(itertools.permutations(s, 2))

In [3]:
#### Function: createPickups ####
## Input: number of orders picked up
## Return: indicies i to n (indicating pickups)
def createPickups(n):
    return [item for item in range(1, n+1)]
#### Function: createDropoffs ####
## Input: number of orders picked up
## Return: indicies n+1 to 2n
def createDropoffs(n):
    return [item for item in range(n+1, 2*n+1)]

In [4]:
#### Function: findLocations ####
## Input: name of json data file
## Return: list of delivery location names 
def findLocations(json):
    locations = []
    for i in json:
        location = json[i]['name']
        locations.append(location)
    return(locations)
 
#### Function: findCoord ####
## Input: name of json data file
## Return: list of coordinate pairs of delivery locations
def findCoord(json):
    coordinates = {}
    for i in json:
        coordinates[i] = (float(json[i]['lat']), float(json[i]['long']))
    return(coordinates)  

In [5]:
#### Function: Edges ###
## Inputs: list of 0, 1-2n, and 2n+1 (V)
## Return: list of possible edges in route
def edges(V,P):
    E = list(findsubsetsL2(V))
    for index, tup in enumerate(E.copy()):
        x = tup[0]
        y = tup[1]
        if x >= y and tup in E:
            E.remove(tup)
        else:
            for p in P:
                if (x == 0 and y == n+p) and tup in E:
                    E.remove(tup)
                elif (x == p and y == (2*n) + 1) and tup in E:
                    E.remove(tup)
    return(E)

In [6]:
#### Function: Distance ####
## Input: name of location 1 and location 2 
## Return: Distances between each of the locations
def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    return(geopy.distance.geodesic(c1, c2).miles)

### Function: distMatrix ####
## Input: distances between each location
## Return: 2n+2 x 2n+2 matrix of distances between each location
def distMatrix(locations):
    dist = [distance(c1, c2) for c1, c2 in itertools.product(locations, repeat=2)]
    c = np.reshape(dist, (2*n+2, 2*n+2)) # Create distance matrix
    return(c)

### Model Inputs: 

In [7]:
# Select Data File
ubereats_json = json.load(open('UberEats Data 7.json'))

# Number of orders in file 
n = 7

In [8]:
P = createPickups(n) # Indexes for pickups
D = createDropoffs(n)# Indexes for dropoffs

# Combine pickups and drop offs with first and last location
V = [0] + P + D + [2*n+1]
E = edges(V,P)

# All subsets in V
S = list(powerset(V))

# Generate location list
locations = findLocations(ubereats_json)
coordinates = findCoord(ubereats_json)

# Create Distance Matrix
c = distMatrix(locations)

### Constraints:

In [9]:
#### Function: Delta ####
## Description: {(i, j) ∈ E : i ∈ S, j not ∈ S or 
## i not ∈ S, j ∈ S} for any set of vertices S ⊆ V
## Input: A subset of V
## Return: List of qualified coordinate pairs 
def delta(S,E):
    Stemp = []
    if not isinstance(S, int):
        for index, tup in enumerate(E):
            i = tup[0]
            j = tup[1]
            if (i in S and j not in S) or (i not in S and j in S):
                Stemp.append(E[index])
            
    else: #S is an int
        for index, tup in enumerate(E):
            i = tup[0]
            j = tup[1]
            if i == S or j == S:
                Stemp.append(E[index])
    return(Stemp)

In [10]:
#### Function: mu ####
## Description: U is the collection of subsets S⊂V satisfying
# 3≤|S|≤|V|−2 with 0 ∈ S, 2n + 1 not ∈ S and for which there
# exists i ∈ P such that i not ∈ S and n + i ∈ S. 
## Input: S- subsets of V, V- list of indicies 0 to 2n+1
## Return: U- qualifiying subsets of listed conditions
def mu(S,V):
    # Orders must be picked up before delivery constraint
    U =[] # the collection of subsets S⊂V satisfying 3≤|S|≤|V|−2 with 0∈S, 2n + 1 not ∈ S 
    for i in range(len(S)):
        if len(S[i]) >= 3 and len(S[i]) <= len(V) - 2 and 0 in S[i] and not (2*n+1) in S[i]:
            U.append(S[i])
        
    # and for which there exists i ∈ P such that i not ∈ S and n + i ∈ S
    U_temp = []
    for u in range(len(U)):
        for i in range(len(P)):
            if not P[i] in U[u] and n+P[i] in U[u]:
                U_temp.append(U[u])
    U = U_temp
    return(U)

In [11]:
#### Function: constraint1 #### 
## Description: Starting location must be revisted 
## Input: number of orders ##
def constraint1(n):
    m.addConstr(x[0,2*n+1] == 1)

In [12]:
#### Function: constraint2 ####
## Description: Each location must be visited only once
# Input: list of location indicies 0 to 2n+1
def constraint2(V,E):
    for i in V:
        d = delta(i,E)
        m.addConstr(sum(x[d[di][0], d[di][1]] for di in range(len(d))) == 2)

In [13]:
#### Function: constraint3 ####
## Description: Subtours are eliminates when subsets of V that are in
## delta(S) have length at least 3, length less than or equal to length of V/2, and 
## the edges in this set visited are greater than or equal to 2 in every qualifying subset
## Input: S- from delta(S) function, V- indices of pickups, deliveries, 0, and 2n+1
def constraint3(S,V,E):
    for i in range(len(S)):
        if len(S[i]) >=3 and len(S[i]) <= len(V)/2:
            constr = delta(S[i],E)
            m.addConstr(sum(x[constr[si][0], constr[si][1]] for si in range(len(constr))) >= 2)      

In [14]:
#### Function: constraint4 ####
## Description: Ensures pickups are picked up before they are delivered
## by ensuring subsets of edges in mu that are >= 3 in length and less than 
## length of V-2 each have at least 4 visited edges
## Input: S- subsets of V, V- indexes 0 to 2n+1
def constraint4(S,V,E):
    U = mu(S,V)
    for u in range(len(U)):
        U_constr = delta(U[u],E)
        m.addConstr(sum(x[U_constr[si][0], U_constr[si][1]] for si in range(len(U_constr))) >= 4)

### Optimization Model:

In [15]:
m = gp.Model("ubereats")

# Keep track of full route
x = m.addVars(V,V, vtype=gp.GRB.BINARY, name = "x")

# Set objective function
m.setObjective(gp.quicksum(x[i, j]*c[i][j]
                for i in V for j in V), GRB.MINIMIZE)

# Add Constraints:
constraint1(n) # Return starting location after last location
constraint2(V,E) # Visit each location only once
constraint3(S,V,E) # Subtour elimination
constraint4(S,V,E) # Pickups before deliveries 


# Optimize model
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 67748 rows, 256 columns and 3674335 nonzeros
Model fingerprint: 0x1397826e
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 35094 rows and 151 columns
Presolve time: 2.49s
Presolved: 32654 rows, 105 columns, 1717527 nonzeros
Variable types: 0 continuous, 105 integer (105 binary)
Found heuristic solution: objective 104.5643574

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0         72    -infinity      7.2550980e+01      4s
       1        109   5.4745893e+01   6.3648436e+01    

### Results:

In [16]:
# Create Tour List
def route(vals):
    selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
    tour = []
    target = 0 
    j = 0
    while j < len(selected):
        for i in range(len(selected)):
            if target in selected[i]:
                i1 = selected[i][0]
                i2 = selected[i][1]
                if i1 not in tour:
                    tour.append(i1)
                    target = i1
                elif i2 not in tour:
                    tour.append(i2)
                    target = i2
                j+=1 

    # Because the tour in a undirected graph, sometimes the above graph may return the loop in the other direction
    # This loop corrects that if it happens
    temp_tour = [0]
    if tour[1] > n: # Checking if the loop is going backwards
        for i in range(len(tour)-1): 
            j = tour[len(tour)-1-i]
            temp_tour.append(j)
    tour = temp_tour

    # Get the location names instead of the numbers:
    route = []
    for i in range(len(tour)):
        route.append(locations[tour[i]])
    route.append(locations[tour[0]]) # Revist starting location
    return(route)

In [17]:
# Print Results
# Objective Value
print("obj_func = ", m.objVal)

# Variables 

for v in m.getVars():
    if v.x > 0.5:
        print('%s = %g' % (v.varName, v.x))       

obj_func =  56.174792624389276
x[0,4] = 1
x[0,15] = 1
x[1,2] = 1
x[1,11] = 1
x[2,6] = 1
x[3,8] = 1
x[3,13] = 1
x[4,5] = 1
x[5,6] = 1
x[7,9] = 1
x[7,11] = 1
x[8,14] = 1
x[9,12] = 1
x[10,14] = 1
x[10,15] = 1
x[12,13] = 1


In [18]:
# Create Tour List
vals = m.getAttr('x', x)
path = route(vals)

In [19]:
# Print Tour
print("Optimal Route:")
print(path)

# Print Distance
print("\nDistance: {} miles".format(round(m.objVal,2)))

# Print time
time = (m.objVal + (4*n))/20 # Assume constant speed of 20 mph
hours = math.floor(time) # Assume 2 minutes to go into store and 2 minutes to drop at door
minutes= math.ceil((time - hours)*60)
print("\nEstimated Time: {} hr {} min".format(hours,minutes))


# Print payout
profit = []
for i in ubereats_json:
    pay = float(ubereats_json[i]['profit'])
    profit.append(pay)
print("\nEstimated Profit: ${}".format(round(sum(profit),2)))

Optimal Route:
['Dunkin Donuts', 'Pickup Sonic Drive-In', 'Pickup Tropical Smoothie Cafe', 'Pickup Starbucks', 'Pickup McDonalds', 'Pickup Capriottis Sandwich Shop', 'Deliver Sonic Drive-In', "Pickup Moe's", 'Deliver McDonalds', 'Deliver Tropical Smoothie Cafe', 'Deliver Starbucks', "Pickup Carl's Jr.", 'Deliver Capriottis Sandwich Shop', "Deliver Moe's", "Deliver Carl's Jr.", 'Subway', 'Dunkin Donuts']

Distance: 56.17 miles

Estimated Time: 4 hr 13 min

Estimated Profit: $42.62


### Map:

In [20]:
# Map Tour 
map = folium.Map(location=[36,-115], zoom_start = 10)

points = []
for stop in path:
    points.append(coordinates[stop])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

# Part II: Ohio State UberEats Optimizer
After learning how to optimize UberEats routes, we wanted to start delivering around Ohio State's campus before class. Our starting location will be Arps garage (where car is parked) and the end location will be Scott Lab E024 where ISE 3230 meets. 

### How many orders do you want to take? : 

In [21]:
## After Entering Number, restart kernel or run collapsed cells below
## Min = 2, Max: 7 
n = 3

In [22]:
## This program uses most of the features of the program above, but will randomly select n orders and n drop offs from 15 resturants and 15 
# drop off locations and optimize the route for shortest time
print("Searching for Orders...")

# Read location names and coordinates from json file
ubereats_columbus = json.load(open('UberEats Columbus.json'))

# Randomly select orders and drop offs
Orders = []
Dropoffs = []
Orders = random.sample(range(1, 16), n)
Dropoffs = random.sample(range(16,31),n)

# Create set of pickups and deliveries 
Orders_Selected = [0] + Orders + Dropoffs + [31]
Orders_Selected.sort()


# Create a list of locations
locations = findLocations(ubereats_columbus)
# Create a list of coordinates 
coordinates = findCoord(ubereats_columbus)


# Create list of profits
profit = []
for i in ubereats_columbus:
    pay = float(ubereats_columbus[i]['profit'])
    profit.append(pay)


# Filter Lists for randomly selected deliveries
# Locations & Profit of selected pickups/dropoffs
L = locations.copy()
P = profit.copy()
for l in range(len(L)):
    location = L[l]
    pay = P[l]
    if not l in Orders_Selected:
        locations.remove(location)
        profit.remove(pay)
        
# Coordinates of selected pickups/dropoffs
counter = 0
for key in ubereats_columbus:
    obj = ubereats_columbus[key]
    if counter not in Orders_Selected:
        coordinates.pop(obj['name'])
    counter = counter + 1

Searching for Orders...


In [23]:
### Model Inputs ####
P = createPickups(n) # Indexes for pickups
D = createDropoffs(n)# Indexes for dropoffs
# Combine pickups and drop offs with first and last location
V = [0] + P + D + [2*n+1]

# All subsets in V
S = list(powerset(V))

# All possible Edges
E = edges(V,P)

# Distance Matrix
c = distMatrix(locations)

print("Orders Available:")
for i in range(1,n+1):
    print("  Devliver {} to {}".format(locations[i], locations[i+n]))

Orders Available:
  Devliver Forno to Ohio Stadium
  Devliver Raising Cane's to The Blackwell
  Devliver Oxley's by the Numbers to OSU Wexner Medical Center


In [24]:
print("Calculating Optimal Route...")

# Run optimization model
# Stop model output from printing
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model("ubereats_cbus", env=env)

# Keep track of full route
x = m.addVars(V,V, vtype=gp.GRB.BINARY, name = "x")

# Set objective function
m.setObjective(gp.quicksum(x[i, j]*c[i][j]
                for i in V for j in V), GRB.MINIMIZE)

# Add Constraints:
constraint1(n) # Return starting location after last location
constraint2(V,E) # Visit each location only once
constraint3(S,V,E) # Subtour elimination
constraint4(S,V,E) # Pickups before deliveries 

# Optimize model
m.optimize()

# Create Optimal Route 
vals = m.getAttr('x', x)
path = route(vals)
print("\nOptimal Route: ")
for j in range(len(path)-1):
    print("  " +path[j])

Calculating Optimal Route...

Optimal Route: 
  Start: Arps Garage
  Oxley's by the Numbers
  Raising Cane's
  Forno
  OSU Wexner Medical Center
  Ohio Stadium
  The Blackwell
  End: Scott Lab E024


In [25]:
print("Calculating Payout...")
print("Estimated Profit: ${}".format(round(sum(profit),2)))

Calculating Payout...
Estimated Profit: $19.61


In [26]:
# Enhance interface 
print("Calculating distance...")
print("Distance: {} miles".format(round(m.objVal,2)))

# Time to order assuming 20 mph speed plus 2 minutes to pickup and drop off orders
print("\nCalculating time...")
time = (m.objVal + (4*n))/20 # Assume constant speed of 20 mph
hours = math.floor(time) # Assume 2 minutes to go into store and 2 minutes to drop at door
mins= math.ceil((time - hours)*60)
print("Estimated Time: {} hr {} min".format(hours,mins))

Calculating distance...
Distance: 4.93 miles

Calculating time...
Estimated Time: 0 hr 51 min


In [28]:
print("Generating Map...")

# Map Tour 
map = folium.Map(location=[40,-83], zoom_start = 13.3)

points = []
for stop in path:
    points.append(coordinates[stop])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

Generating Map...
